In [ ]:
import tensorflow as tf

import os
import math
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tqdm import tqdm

for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
from tensorflow.keras.utils import Sequence


def scaling(input_image):
    input_image = input_image / 255.0
    return input_image


class Dataloader(Sequence):

    def __init__(self, x_set, y_set, batch_size, shuffle = False):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        
        batch_x = [self.x[i] for i in indices]
        batch_y = [self.y[i] for i in indices]
        
        input_image = np.array([
            scaling(img_to_array(load_img(file_name))) for file_name in batch_x])
        truth_image = np.array([
            scaling(img_to_array(load_img(file_name))) for file_name in batch_y])
        
        height = input_image.shape[1]
        width = input_image.shape[2]
    
        input_patch_size = 64
        scale = 4
        
        # randomly crop
        truth_patch_size = input_patch_size * scale
        
        input_x = np.random.randint(width - input_patch_size)
        input_y = np.random.randint(height - input_patch_size)
        truth_x = input_x * scale
        truth_y = input_y * scale
        
        input_patch = input_image[:, input_y:(input_y+input_patch_size), input_x:(input_x+input_patch_size)]
        truth_patch = truth_image[:, truth_y:(truth_y+truth_patch_size), truth_x:(truth_x+truth_patch_size)]
        
        # randomly rotate
        rot90_k = np.random.randint(4)+1
        input_patch = np.rot90(input_patch, k=rot90_k, axes=(1, 2))
        truth_patch = np.rot90(truth_patch, k=rot90_k, axes=(1, 2))
        
        # randomly flip
        flip = (np.random.uniform() < 0.5)
        if (flip):
            input_patch = input_patch[:, :, ::-1]
            truth_patch = truth_patch[:, :, ::-1]
            
        # finalize
        return input_patch, truth_patch

    def on_epoch_end(self):
        self.indices = np.arange(len(self.x))
        if self.shuffle == True:
            np.random.shuffle(self.indices)
            
            
train_blur_dir = "./NTIRE/LR/train_blur_bicubic/train/train_blur_bicubic/X4/"
train_blur = []
idx=240
for folder in tqdm(range(0,idx)):
    if folder<=9:
        temp = train_blur_dir +("00"+str(folder)+"/")
    elif folder <=99:
        temp = train_blur_dir +("0"+str(folder)+"/")
    else:
        temp = train_blur_dir +(str(folder)+"/")
    for file in range(0,100):
        
        if file <=9:
            temp_2 = temp + str("0000000")+str(file)+".png"
                
        else:
            temp_2 = temp + str("000000")+str(file)+".png"
        
        train_blur.append(temp_2)

train_target_dir = "./NTIRE/LR/train_sharp/train/train_sharp/"
train_target = []
for folder in tqdm(range(0,idx)):
    if folder<=9:
        temp = train_target_dir +("00"+str(folder)+"/")
    elif folder <=99:
        temp = train_target_dir +("0"+str(folder)+"/")
    else:
        temp = train_target_dir +(str(folder)+"/")
    for file in range(0,100):
        if file <=9:
            temp_2 = temp + str("0000000")+str(file)+".png"
                
        else:
            temp_2 = temp + str("000000")+str(file)+".png"
        
        train_target.append(temp_2)
        

val_blur_dir = "./NTIRE/LR/val_blur_bicubic/val/val_blur_bicubic/X4/"
val_blur = []
for folder in tqdm(range(0,30)):
    if folder%10==9:
        if folder<=9:
            temp = val_blur_dir +("00"+str(folder)+"/")
        elif folder <=99:
            temp = val_blur_dir +("0"+str(folder)+"/")
        else:
            temp = val_blur_dir +(str(folder)+"/")
        for file in range(0,100):
        
            if file <=9:
                temp_2 = temp + str("0000000")+str(file)+".png"
                
            else:
                temp_2 = temp + str("000000")+str(file)+".png"
            
            val_blur.append(temp_2)

val_target_dir = "./NTIRE/LR/val_sharp/val/val_sharp/"
val_target = []
for folder in tqdm(range(0,30)):
    if folder%10==9:
        if folder<=9:
            temp = val_target_dir +("00"+str(folder)+"/")
        elif folder <=99:
            temp = val_target_dir +("0"+str(folder)+"/")
        else:
            temp = val_target_dir +(str(folder)+"/")
        for file in range(0,100):
            
            if file <=9:
                temp_2 = temp + str("0000000")+str(file)+".png"
                
            else:
                temp_2 = temp + str("000000")+str(file)+".png"
        
            val_target.append(temp_2)


train_loader = Dataloader(train_blur, train_target, 8, shuffle=True)
valid_loader = Dataloader(val_blur, val_target, 8)


In [ ]:
def get_model(upscale_factor=2, channels=3):
    conv_args = {
        "activation": "relu",
        "kernel_initializer": keras.initializers.glorot_normal(seed=None),
        "padding": "same"
    }
    conv_args2 = {
        "kernel_initializer": keras.initializers.glorot_normal(seed=None),
        "padding": "same"
    }
    
    inputs = keras.Input(shape=(None, None, channels))
    
    
    #conv
    x1 = layers.Conv2D(filters=128, kernel_size=(9,9),**conv_args)(inputs)
    
    x = x1
    
    #block
    repetition= 12
    for i in range(0,repetition):
        r_block = x
        x_2 = layers.Conv2D(filters=128, kernel_size=(3,3),**conv_args)(x)
        
        #attention
        sobel = tf.image.sobel_edges(x_2)
        sobel = tf.math.abs(sobel[:, :, :, :, 0]) + tf.math.abs(sobel[:, :, :, :, 1])
        
        sobel = layers.Conv2D(filters=128, kernel_size=(3,3),**conv_args)(sobel)
        sobel = layers.Conv2D(filters=128, kernel_size=(3,3),**conv_args2)(sobel)
        sobel = keras.activations.sigmoid(sobel)
        x = keras.layers.Multiply()([x_2,sobel])
        
        #atrous conv
        ar_1 = layers.Conv2D(filters=64, kernel_size=(3,3),**conv_args,dilation_rate=1)(x_2)
        ar_3 = layers.Conv2D(filters=64, kernel_size=(3,3),**conv_args,dilation_rate=3)(x_2)
        ar_5 = layers.Conv2D(filters=64, kernel_size=(3,3),**conv_args,dilation_rate=5)(x_2)
        ar = tf.concat([ar_1,ar_3,ar_5],axis=-1)
        
        ar = layers.Conv2D(filters=128, kernel_size=(3,3),**conv_args,dilation_rate=1)(ar)
        
        ar_1 = layers.Conv2D(filters=64, kernel_size=(3,3),**conv_args,dilation_rate=1)(ar)
        ar_3 = layers.Conv2D(filters=64, kernel_size=(3,3),**conv_args,dilation_rate=3)(ar)
        ar_5 = layers.Conv2D(filters=64, kernel_size=(3,3),**conv_args,dilation_rate=5)(ar)
        ar = tf.concat([ar_1,ar_3,ar_5],axis=-1)
        
        
        #concat attention & atrous
        x = tf.concat([x,ar],axis=-1)
        
        #conv
        x = layers.Conv2D(filters=128, kernel_size=(3,3),**conv_args2)(x)
        
        # residual
        x = x + r_block
    
    # concat
    x = tf.concat([x,x1],axis=-1)
    
    #conv
    x = layers.Conv2D(filters=128,kernel_size=(3,3),**conv_args)(x)
    
    #ESPCN
    x = layers.Conv2D(64, 5, **conv_args)(x)
    x = layers.Conv2D(64, 3, **conv_args)(x)
    x = layers.Conv2D(32, 3, **conv_args)(x)
    
    x = layers.Conv2D(32 * (upscale_factor ** 2), 3, **conv_args2)(x)
    x = tf.nn.depth_to_space(x, upscale_factor)
    x = layers.Conv2D(32 * (upscale_factor ** 2), 3, **conv_args2)(x)
    x = tf.nn.depth_to_space(x, upscale_factor)
    
    outputs = layers.Conv2D(3, 3, **conv_args2)(x)
    
    #skip branch
    x_skip = tf.image.resize(inputs,[tf.shape(inputs)[1]*4,tf.shape(inputs)[2]*4],method='bicubic')
    
    outputs = outputs + x_skip
    
    
    return keras.Model(inputs, outputs)

In [ ]:
model = get_model()
model.summary()

In [ ]:
loss_fn = keras.losses.MeanAbsoluteError()

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=200000,
    decay_rate=0.5,
    staircase=True)

optimizer = keras.optimizers.Adam(learning_rate=lr_schedule,beta_1=0.9, beta_2=0.999, epsilon= 1e-8)
epochs = 133
model.compile(
    optimizer=optimizer, loss=loss_fn
)

import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = './save/1/weights.{epoch:02d}.hdf5'
checkpoint = ModelCheckpoint(filepath,             # file명을 지정합니다
                             monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=False,  # 가장 best 값만 저장합니다
                             save_weights_only=False,
                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max
                            )
model.fit(train_loader, validation_data=valid_loader,batch_size=8, epochs=epochs,callbacks=[checkpoint,tensorboard_callback])

In [ ]:
class Dataloader(Sequence):

    def __init__(self, x_set, y_set, batch_size, shuffle = False):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        
        batch_x = [self.x[i] for i in indices]
        batch_y = [self.y[i] for i in indices]
        
        input_image = np.array([
            scaling(img_to_array(load_img(file_name))) for file_name in batch_x])
        truth_image = np.array([
            scaling(img_to_array(load_img(file_name))) for file_name in batch_y])
        
        height = input_image.shape[1]
        width = input_image.shape[2]
    
        input_patch_size = 128
        scale = 4
        
        # randomly crop
        truth_patch_size = input_patch_size * scale
        
        input_x = np.random.randint(width - input_patch_size)
        input_y = np.random.randint(height - input_patch_size)
        truth_x = input_x * scale
        truth_y = input_y * scale
        
        input_patch = input_image[:, input_y:(input_y+input_patch_size), input_x:(input_x+input_patch_size)]
        truth_patch = truth_image[:, truth_y:(truth_y+truth_patch_size), truth_x:(truth_x+truth_patch_size)]
        
        # randomly rotate
        rot90_k = np.random.randint(4)+1
        input_patch = np.rot90(input_patch, k=rot90_k, axes=(1, 2))
        truth_patch = np.rot90(truth_patch, k=rot90_k, axes=(1, 2))
        
        # randomly flip
        flip = (np.random.uniform() < 0.5)
        if (flip):
            input_patch = input_patch[:, :, ::-1]
            truth_patch = truth_patch[:, :, ::-1]
            
        # finalize
        return input_patch, truth_patch

    def on_epoch_end(self):
        self.indices = np.arange(len(self.x))
        if self.shuffle == True:
            np.random.shuffle(self.indices)

train_loader = Dataloader(train_blur, train_target, 2, shuffle=True)
valid_loader = Dataloader(val_blur, val_target, 2)

In [ ]:
model = keras.models.load_model('./save/1/weights.133.hdf5')

In [ ]:
loss_fn = keras.losses.MeanAbsoluteError()

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=(1e-4)/4,
    decay_steps=200000,
    decay_rate=0.5,
    staircase=True)

optimizer = keras.optimizers.Adam(learning_rate=lr_schedule,beta_1=0.9, beta_2=0.999, epsilon= 1e-8)
epochs = 33
model.compile(
    optimizer=optimizer, loss=loss_fn
)

import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = './save/2/weights.{epoch:02d}.hdf5'
checkpoint = ModelCheckpoint(filepath,             # file명을 지정합니다
                             monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=False,  # 가장 best 값만 저장합니다
                             save_weights_only=False,
                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max
                            )
model.fit(train_loader, validation_data=valid_loader,batch_size=2, epochs=epochs,callbacks=[checkpoint,tensorboard_callback])

In [ ]:
class Dataloader(Sequence):

    def __init__(self, x_set, y_set, batch_size, shuffle = False):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        
        batch_x = [self.x[i] for i in indices]
        batch_y = [self.y[i] for i in indices]
        
        input_image = np.array([
            scaling(img_to_array(load_img(file_name))) for file_name in batch_x])
        truth_image = np.array([
            scaling(img_to_array(load_img(file_name))) for file_name in batch_y])
        
        height = input_image.shape[1]
        width = input_image.shape[2]
    
        input_patch_size = 180
        scale = 4
        
        # randomly crop
        truth_patch_size = input_patch_size * scale
        
        input_x = np.random.randint(width - input_patch_size)
        input_y = 0
        truth_x = input_x * scale
        truth_y = input_y * scale
        
        input_patch = input_image[:, input_y:(input_y+input_patch_size), input_x:(input_x+input_patch_size)]
        truth_patch = truth_image[:, truth_y:(truth_y+truth_patch_size), truth_x:(truth_x+truth_patch_size)]
        
        # randomly rotate
        rot90_k = np.random.randint(4)+1
        input_patch = np.rot90(input_patch, k=rot90_k, axes=(1, 2))
        truth_patch = np.rot90(truth_patch, k=rot90_k, axes=(1, 2))
        
        # randomly flip
        flip = (np.random.uniform() < 0.5)
        if (flip):
            input_patch = input_patch[:, :, ::-1]
            truth_patch = truth_patch[:, :, ::-1]
            
        # finalize
        return input_patch, truth_patch

    def on_epoch_end(self):
        self.indices = np.arange(len(self.x))
        if self.shuffle == True:
            np.random.shuffle(self.indices)

train_loader = Dataloader(train_blur, train_target, 1, shuffle=True)
valid_loader = Dataloader(val_blur, val_target, 1)

In [ ]:
model = keras.models.load_model('./save/2/weights.133.hdf5')

In [ ]:
loss_fn = keras.losses.MeanAbsoluteError()

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=(1e-4)/16,
    decay_steps=200000,
    decay_rate=0.5,
    staircase=True)

optimizer = keras.optimizers.Adam(learning_rate=lr_schedule,beta_1=0.9, beta_2=0.999, epsilon= 1e-8)
epochs = 11
model.compile(
    optimizer=optimizer, loss=loss_fn
)

import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = './save/3/weights.{epoch:02d}.hdf5'
checkpoint = ModelCheckpoint(filepath,             # file명을 지정합니다
                             monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=False,  # 가장 best 값만 저장합니다
                             save_weights_only=False,
                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max
                            )
model.fit(train_loader, validation_data=valid_loader,batch_size=1, epochs=epochs,callbacks=[checkpoint,tensorboard_callback])